## Use OpenCL To Add Two Random Arrays (This Way Shows Details)

In [27]:
import pyopencl as cl
import numpy as np 

In [28]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

a = np.random.rand(50000).astype(np.float32)
b = np.random.rand(50000).astype(np.float32)
c = np.empty_like(a)

a_gpu = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf = a)
b_gpu = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf = b)
c_gpu = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, c.nbytes)


In [29]:
kernel = """__kernel void sum(__global const float *a, __global const float *b, __global float *c) {
    int i = get_global_id(0);
    c[i] = a[i] + b[i];
    }   """
program  = cl.Program(context, kernel).build()
program.sum(queue, a.shape, None, a_gpu, b_gpu, c_gpu)

np_array = [a, b, c]
gpu_array = [a_gpu, b_gpu, c_gpu]

for x in range(3):
    cl.enqueue_copy(queue, np_array[x], gpu_array[x])
queue.finish()

for x in np_array:
    print(x)

[0.5257073  0.23750056 0.11271624 ... 0.7317663  0.52242005 0.7100476 ]
[0.8451233  0.8651082  0.49618682 ... 0.16361749 0.27418095 0.6738464 ]
[1.3708305  1.1026088  0.60890305 ... 0.8953838  0.796601   1.383894  ]
